In [ ]:
#imports 
import json
import os
import requests
import pandas as pd
import matplotlib as plt
import pprint as pprint
from pandas.io.json import json_normalize

import gmaps
import gmaps.datasets



In [ ]:
# Remember to update the config file with your API key
from config import api_key
from config import api_id
from config import gkey

In [ ]:
url = "https://api.adzuna.com/v1/api/jobs/us/search/"
#have to create iteration to get all pages of data
pagetest = "1"
api_details ="?" + "app_id=" + api_id + "&app_key=" + api_key
# Build query URL
query_url = url + pagetest + api_details
query_url

In [ ]:
data_response = requests.get(query_url)
data_json = data_response.json()
type(data_json)


In [ ]:

pages = [str(x) for x in range(1,1001)]


title = []
company_name = []
location = []
post_created = []
category = []
lat = []
lng = []

num = 0
nested_num = 0

for page in pages:
        response = requests.get(url + page + api_details).json()
        num += 1 
        for index_num in range(10):
            try:
                title.append(response['results'][index_num]['title'])
                company_name.append(response['results'][index_num]['company']['display_name'])
                location.append(response['results'][index_num]['location']['display_name'])
                post_created.append(response['results'][index_num]['created'])
                category.append(response['results'][index_num]['category']['label'])
                lat.append(response['results'][index_num]['latitude'])
                lng.append(response['results'][index_num]['longitude'])
                nested_num += 1
            except KeyError:
                if len(title) > len(lng):
                    title.pop(-1)
                    if len(company_name) > len(title):
                        company_name.pop(-1)
                        if len(location) > len(company_name):
                            location.pop(-1)
                            if len(post_created) > len(location):
                                post_created.pop(-1)
                                if len(category) > len(post_created):
                                    category.pop(-1)
                                    if len(lat) > len(category):
                                        lat.pop(-1)
                                        
                if len(lng) == len(lat) and len(lng) == len(category) and len(lng) == len(post_created) and len(lng) == len(location) and len(lng) == len(company_name) and len(lng) == len(title):
                    print(f'Missing attribute within column for Job Post on page {num} . Removed Job Post Row ...')
                    
        print(f"page {num} completed. Iterrated through {nested_num} job postings ")
            
        

In [ ]:
#checking list to see if they are the same length
len(title)
len(company_name)
len(location)
len(post_created)
len(category)
# # #lat has 
len(lat)
len(lng)

In [ ]:
job_posting_dict = {
    
    'Title':title,
    'Company Name':company_name,
    'Location':location,
    'Job Post Created':post_created,
    'Category':category,
    'Lat':lat,
    'Lng':lng
}

test_job_posting_df = pd.DataFrame(job_posting_dict)
test_job_posting_df.count()

In [ ]:
#Export to a CSV file
test_job_posting_df.to_csv('./Resources/test_data_10000.csv')

In [ ]:
#testing to remove appending
cities = ["Hamburg", "Linz", "Salzburg", "Vienna"]
cities.pop(-1)
cities

In [ ]:
#Alternative way to get data using json_normalize
#LINK: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html

pages_test = [str(x) for x in range(1,101)]

counter = 0
# Build query URL
data_pages_df=pd.DataFrame()

for page_num in pages_test:
    response = requests.get(url + page_num + api_details).json()
    
    #create a df to store the normalized page that has the job postings
    normalized_page = json_normalize(response['results'])
    
    #add normalized data into a df
    data_page_df=pd.DataFrame(normalized_page)
    
    #add the url column to df incase an error occurs you know what page you were on
    data_page_df['query_url'] = 'page number ' + page_num
    
    #append to a new df so each page can be safed
    data_pages_df =data_pages_df.append(data_page_df)
    
    counter += 1
    print(f"Page {page_num} iteration complete")
    
print(f'total rows iterated | {counter * 10}')   
      
#data_pages_df=drop.na((latitude))




In [ ]:
#save alternative way
data_pages_df.to_csv('./Resources/normalized_dataframe.csv')

In [ ]:
#Cleaning up data for Alternative way
#data_pages_df=drop.na((latitude))

In [ ]:
#calling the saved df
filepath = os.path.join(".","Resources", "test_data_10000.csv")
filepath

read_file = pd.read_csv(filepath)
read_file.head()


In [ ]:
gmaps.configure(api_key= gkey) # Fill in with your API key
job_data_copy = read_file.copy()

lat = job_data_copy['Lat']
lng = job_data_copy['Lng']
cat = job_data_copy['Unnamed: 0']


cat = cat.astype('float')

In [ ]:
locations = job_data_copy[['Lat', 'Lng']]
fig = gmaps.figure(map_type='TERRAIN')



locations_layer = gmaps.symbol_layer(
    locations, fill_color='red', stroke_color='green', scale=1
)
fig = gmaps.figure()
fig.add_layer(locations_layer)
fig

In [ ]:
charr = job_data_copy['Category'].unique()
charr

In [ ]:
#category
index_cat = job_data_copy.set_index(['Category'])
cat = index_cat.loc["Sales Jobs"] 
healthcare = cat[['Lat', 'Lng']]
healthcare

# for c in range(len(charr)):
#     flag = index_cat.loc[c]
#     arm = flag[['Lat','Lng']]
    
job_data_copy